In [443]:
import pandas as pd
import math
from math import sqrt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# read in data predictions for BTC

In [444]:
predictions_df = pd.read_csv("training_configs/btc_all_predictions.csv", parse_dates=True)
true_price_df = pd.read_csv("../tmp/historic_crypto_prices - bitcoin_jan_2017_sep_4_2021 copy.csv")

In [445]:
predictions_df.head()

,date,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,date_prediction_for,test_model_lookback_1
0,2010-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010-01-09,0.0
1,2019-04-02,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,2019-04-09,0.0
2,2019-04-03,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,2019-04-10,0.0
3,2019-04-04,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,2019-04-11,0.0
4,2019-04-05,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,2019-04-12,0.0


In [446]:
predictions_df.date.min()

'2010-01-01'

In [447]:
predictions_df.date.max()

'2020-04-22'

In [448]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 9 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   date                                                    388 non-null    object 
 1   nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2  388 non-null    float64
 2   tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2     388 non-null    float64
 3   nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2  388 non-null    float64
 4   tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2     388 non-null    float64
 5   nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2  388 non-null    float64
 6   tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2     388 non-null    float64
 7   date_prediction_for                                     388 non-null    object 
 8   test_model_lookback_1                    

In [449]:
true_close_df = true_price_df[['date','close']]

### Map the predictions for date to the true price date

In [450]:
predictions_df.date_prediction_for

0       2010-01-09
1       2019-04-09
2       2019-04-10
3       2019-04-11
4       2019-04-12
          ...     
383     2020-04-25
384     2020-04-26
385     2020-04-27
386     2020-04-28
387     2020-04-29
Name: date_prediction_for, Length: 388, dtype: object

In [451]:
merged_df = pd.merge(predictions_df, true_close_df, left_on='date_prediction_for',
                     right_on ='date', suffixes=['_pred','_true'])

In [452]:
merged_df['day'] = [t.day for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['month'] =  [t.month for t in pd.to_datetime(merged_df.date_prediction_for)]
merged_df['quarter'] = [t.quarter for t in pd.to_datetime(merged_df.date_prediction_for)]

In [453]:
merged_df.drop(['date_pred', 'date_true', 'date_prediction_for'], inplace=True, axis=1)

In [454]:
merged_df.tail()

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter
382,7012.630304,6811.581457,6917.758932,6736.856893,6106.502534,5939.083995,0.0,7569.94,25,4,2
383,6763.870678,6659.527999,6823.889725,6788.286992,6714.833511,6952.502250,0.0,7679.87,26,4,2
384,6836.866651,6252.074488,7089.521509,6618.742987,7035.405521,6891.651539,0.0,7795.60,27,4,2
385,6819.752132,6241.436474,6755.944242,6093.158180,6776.603432,6569.586325,0.0,7807.06,28,4,2
386,6906.912407,6861.025597,7049.882023,6959.482193,6635.017877,6792.484413,0.0,8801.04,29,4,2


In [455]:
merged_df.describe()

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,close,day,month,quarter
count,387.000000,387.000000,387.000000,387.000000,387.000000,387.000000,387.0,387.000000,387.000000,387.000000,387.000000
mean,8708.982634,8719.058091,8803.387452,8967.054407,8908.432537,8599.422788,0.0,8470.908811,15.932817,6.377261,2.478036
std,1961.564820,4609.162935,2167.541133,4427.360053,2318.820490,4623.030728,0.0,1716.765369,8.726746,3.408660,1.094692
min,4546.248123,139.877840,3666.543647,-289.448271,3261.251466,-280.599843,0.0,4970.790000,1.000000,1.000000,1.000000
25%,7150.902190,6028.148384,7337.444302,6560.256340,7410.219397,6560.265697,0.0,7291.545000,9.000000,4.000000,2.000000
50%,8712.896702,8658.079123,8828.010997,8641.864848,8712.027292,8541.643228,0.0,8367.850000,16.000000,6.000000,2.000000
75%,10192.858222,10683.591729,10270.955676,10980.669525,10533.108686,10134.207317,0.0,9742.110000,23.000000,9.000000,3.000000
max,13331.237899,23654.633486,14332.008433,23157.885335,14228.875403,23173.074402,0.0,13016.230000,31.000000,12.000000,4.000000


### train test split

In [459]:
split_pct = .80
l_merged = len(merged_df)
merged_df_train, merged_df_test = merged_df.iloc[:int(split_pct*l_merged),:],  merged_df.iloc[int(l_merged*split_pct):,:]
                                                                                                  

In [460]:
print(f"train rows = {len(merged_df_train)}")

train rows = 309


In [461]:
print(f"test rows = {len(merged_df_test)}")

test rows = 78


In [462]:
merged_df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 308
Data columns (total 11 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2  309 non-null    float64
 1   tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2     309 non-null    float64
 2   nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2  309 non-null    float64
 3   tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2     309 non-null    float64
 4   nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2  309 non-null    float64
 5   tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2     309 non-null    float64
 6   test_model_lookback_1                                   309 non-null    float64
 7   close                                                   309 non-null    float64
 8   day                                     

In [463]:
merged_df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78 entries, 309 to 386
Data columns (total 11 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2  78 non-null     float64
 1   tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2     78 non-null     float64
 2   nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2  78 non-null     float64
 3   tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2     78 non-null     float64
 4   nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2  78 non-null     float64
 5   tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2     78 non-null     float64
 6   test_model_lookback_1                                   78 non-null     float64
 7   close                                                   78 non-null     float64
 8   day                                    

In [464]:
merged_df_train.iloc[:, ~merged_df.columns.isin(['close'])]

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,day,month,quarter
0,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,9,4,2
1,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,10,4,2
2,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,11,4,2
3,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,12,4,2
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,13,4,2
...,...,...,...,...,...,...,...,...,...,...
304,9214.560033,8638.859278,9514.803447,8747.076239,8678.583061,7754.460725,0.0,7,2,1
305,8978.097522,9232.719568,9555.716547,8643.459417,9266.712913,8586.732716,0.0,8,2,1
306,8794.895398,8859.047150,8823.665832,8445.710527,8552.933125,8118.653406,0.0,9,2,1
307,9389.062584,8380.000666,8928.207805,8332.387134,9077.974163,9458.397391,0.0,10,2,1


In [465]:


merged_df_x_train, merged_df_y_train = merged_df_train.iloc[:, ~merged_df_train.columns.isin(['close'])], merged_df_train.iloc[:, merged_df_train.columns.isin(['close'])]
merged_df_x_test, merged_df_y_test =  merged_df_test.iloc[:, ~merged_df_test.columns.isin(['close'])], merged_df_test.iloc[:, merged_df_test.columns.isin(['close'])]
                                                                                                                                                 

In [466]:
merged_df_x_train

,nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2,nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2,test_model_lookback_1,day,month,quarter
0,4752.149527,253.537135,4730.976374,654.310204,3469.734359,795.201430,0.0,9,4,2
1,4546.248123,407.206977,4373.560445,255.627627,4437.900633,1413.509939,0.0,10,4,2
2,4550.998590,726.168584,4045.591506,1861.981408,4012.011230,523.093010,0.0,11,4,2
3,5003.264197,353.606543,4819.668698,817.215937,3977.238773,553.057301,0.0,12,4,2
4,5184.268504,197.768508,5123.185573,1596.333494,4093.602236,920.106602,0.0,13,4,2
...,...,...,...,...,...,...,...,...,...,...
304,9214.560033,8638.859278,9514.803447,8747.076239,8678.583061,7754.460725,0.0,7,2,1
305,8978.097522,9232.719568,9555.716547,8643.459417,9266.712913,8586.732716,0.0,8,2,1
306,8794.895398,8859.047150,8823.665832,8445.710527,8552.933125,8118.653406,0.0,9,2,1
307,9389.062584,8380.000666,8928.207805,8332.387134,9077.974163,9458.397391,0.0,10,2,1


In [467]:
merged_df_y_train

,close
0,5204.96
1,5324.55
2,5064.49
3,5089.54
4,5096.59
...,...
304,9795.94
305,9865.12
306,10116.67
307,9856.61


### performance, rmse , on average prediction

In [468]:
average_predictions_train = np.mean(merged_df_x_train.iloc[:,:6], axis=1)
average_predictions_test = np.mean(merged_df_x_test.iloc[:,:6], axis=1)

In [469]:
average_predictions_train[:10]

0    2442.651505
1    2572.342291
2    2619.974055
3    2587.341908
4    2852.544153
5    2721.518844
6    3153.856190
7    2727.367343
8    3387.347636
9    2872.964842
dtype: float64

In [470]:
merged_df_y_train.head(10)

,close
0,5204.96
1,5324.55
2,5064.49
3,5089.54
4,5096.59
5,5167.72
6,5067.11
7,5235.56
8,5251.94
9,5298.39


In [471]:
average_rmse_train = sqrt(mean_squared_error(average_predictions_train, merged_df_y_train))
average_rmse_test = sqrt(mean_squared_error(average_predictions_test, merged_df_y_test))

In [472]:
print(f" The average training rmse is {average_rmse_train}")
print(f" The average testing rmse is {average_rmse_test}")

 The average training rmse is 2126.531016156906
 The average testing rmse is 1269.5154631822002


# train linear regression on ALL predictions

In [473]:
lregr = LinearRegression()

In [474]:
lregr.fit(merged_df_x_train, merged_df_y_train)

LinearRegression()

In [475]:
train_predictions_lrgr = lregr.predict(merged_df_x_train)

In [476]:
test_predictions_lrgr = lregr.predict(merged_df_x_test)

In [477]:
test_predictions_lrgr[:15]


array([[ 9530.31302609],
       [ 9641.21814256],
       [ 9750.63937843],
       [ 9575.0727088 ],
       [ 9613.27930944],
       [10140.66513696],
       [ 9784.97040042],
       [10109.87083714],
       [10045.24976314],
       [ 9679.79011233],
       [10200.77086137],
       [10354.76666947],
       [ 9881.29163348],
       [ 9981.89545256],
       [ 9937.44688711]])

In [478]:
merged_df_y_test.head(15)

,close
309,10326.05
310,10214.38
311,10312.12
312,9889.42
313,9934.43
314,9690.14
315,10142.00
316,9633.39
317,9608.48
318,9686.44


In [479]:
average_rmse_train_lrgr = sqrt(mean_squared_error(train_predictions_lrgr, merged_df_y_train))
average_rmse_test_lrgr = sqrt(mean_squared_error(test_predictions_lrgr, merged_df_y_test))

In [480]:
print(f" The average training rmse with linear stacking is {average_rmse_train_lrgr}")
print(f" The average testing rmse with linear stacking is {average_rmse_test_lrgr}")

 The average training rmse with linear stacking is 871.1178213563562
 The average testing rmse with linear stacking is 1705.1705062697206




#### feature importance

In [481]:
lregr.coef_.ravel()

array([ 2.28479412e-01,  1.29547212e-02,  1.68877915e-01,  3.30203224e-02,
        2.41367111e-01, -7.03849983e-03,  3.35376171e-12, -7.51503287e+00,
        1.47759903e+02, -8.76552251e+02])

In [482]:
lregr.coef_.ravel()[np.argsort(lregr.coef_.ravel())[::-1]]

array([ 1.47759903e+02,  2.41367111e-01,  2.28479412e-01,  1.68877915e-01,
        3.30203224e-02,  1.29547212e-02,  3.35376171e-12, -7.03849983e-03,
       -7.51503287e+00, -8.76552251e+02])

In [483]:
merged_df_x_train.columns[np.argsort(lregr.coef_)[::-1]]

array([['quarter', 'day',
        'tcn_btc_lookback_45_window_5_std_1.25_num_add_dfs_2',
        'test_model_lookback_1',
        'tcn_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
        'tcn_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
        'nbeats_btc_lookback_30_window_5_std_1.25_num_add_dfs_2',
        'nbeats_btc_lookback_15_window_5_std_1.25_num_add_dfs_2',
        'nbeats_btc_lookback_45_window_5_std_1.25_num_add_dfs_2',
        'month']], dtype=object)

## Test Random Forest

In [484]:

rf = RandomForestRegressor(n_estimators=500)

In [485]:
rf.fit(merged_df_x_train, merged_df_y_train)

/var/folders/0r/jkhwtgvx0wb9vrk6xn4bmz2m0000gn/T/ipykernel_16211/1792525404.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(merged_df_x_train, merged_df_y_train)


RandomForestRegressor(n_estimators=500)

In [486]:
train_predictions_rf = rf.predict(merged_df_x_train)
test_predictions_rf = rf.predict(merged_df_x_test)

In [487]:
test_predictions_rf[:15]


array([ 9566.11582,  9795.00402,  9341.8666 ,  9583.92608,  9802.01816,
       10006.83358,  9462.0319 , 10533.72266, 10647.10462,  9294.06816,
       10040.0903 , 10775.6026 , 10018.7682 , 10586.7258 , 10114.48966])

In [488]:
merged_df_y_test.head(15)

,close
309,10326.05
310,10214.38
311,10312.12
312,9889.42
313,9934.43
314,9690.14
315,10142.00
316,9633.39
317,9608.48
318,9686.44


In [489]:
average_rmse_train_rf = sqrt(mean_squared_error(train_predictions_rf, merged_df_y_train))
average_rmse_test_rf = sqrt(mean_squared_error(test_predictions_rf, merged_df_y_test))

In [490]:
print(f" The average training rmse with RF stacking is {average_rmse_train_rf}")
print(f" The average testing rmse with RF stacking is {average_rmse_test_rf}")

 The average training rmse with RF stacking is 218.32306630895167
 The average testing rmse with RF stacking is 1727.0715662803564
